# <center> TP 3 : Apprentissage d'un mélange - Algorithme EM

## Chargement des données MNIST

In [ ]:
%pylab
%matplotlib inline 

from mnist import load_mnist
import numpy as np
import visualize as vz
import random as rd

train_data, train_labels = load_mnist(dataset='training', path='./')
test_data, test_labels = load_mnist(dataset='testing', path='./')

Transformation de `train_data` et `test_data` en vecteur colonne pour chaque exemple.

In [ ]:
train_data = np.reshape(train_data, (60000, 28 * 28)).T
test_data  = np.reshape(test_data,  (10000, 28 * 28)).T

Binarisation des données :

In [ ]:
binarize = lambda x : x < 128 / 255
vfunc = np.vectorize(binarize)
train_data = vfunc(train_data)
test_data = vfunc(test_data)

vz.plotGroupImages(train_data[:, :10])

## Partie 1 : Théorie

A partir de ce qui a été vu en cours, déduire les étapes de l'algorithme EM pour l'apprentissage des paramètres d'un mélange de loi de Bernoulli. Donner les formules de mise à jour des différents paramètres du modèle.

L'algorithme EM fonctionne suivant deux étapes :
* L'étape appelée "expectation", qui consiste à calculer tous les poids de tous les points pour toutes les composantes.
* L'étape appelée "maximization", qui vise à recalculer les moyennes pour chaque composante, en modifiant les poids de chaque composante.

On pose N correspondant aux nombres d'éléments dans le dataset {$X_1, \ldots, X_n$}, K correspondant aux nombres de composantes souhaitées,
$w_i$ le poid associé à la i-ème composante.

**Etape 1 : Initialisation des paramètres**

* Initialisation des moyennes de chaque composante.
* Initialisation des poids, tels que $\sum_{i=1}^{K}{w_i} = \sum_{i=1}^K{P(i)} = 1$.

**Etape 2 : Expectation**

* Nous savons que la vraisemblance $P(X_i | \mu_k) = \mu_k^{X_i} (1 − \mu_k)^{N − X_i}$ en utilisant une distribution de Bernoulli.
* On calcule l'a posteriori : $P(k|X_i) = \frac{w_k P(X_i|\mu_k)}{\sum_{j=1}^{K}{w_j P(X_j|\mu_k)}} = \frac{w_k (\mu_k^{X_i} (1 − \mu_k)^{N − X_i})}{\sum_{j=1}^{K}{w_j (\mu_k^{X_j}(1 − \mu_k)^{N − X_j})}}$

**Etape 3 : Maximization**

* On calcule de nouveaux poids : $P(k) = w_k = \frac{\sum_{i=1}^{N}{P(k|X_i)}}{N}$
* On calcule les nouvelles moyennes : $\mu_k = \frac{\sum_{i=1}^{N}{P(k|X_i)X_i}}{\sum_{i=1}^{N}{P(k|X_i)}}$

## Partie 2 : Implémentation

On vous demandera de tester différentes valeurs pour le nombre de composantes du mélange ainsi que différentes initialisations du modèle.

**1.** Implémenter cet algorithme et faire l'apprentissage sur les données binaires TRAINB obtenues précedemment. Attention à la manipulation des valeurs de probabilités (entre 0 et 1) qui deviennent rapidement nulles à cause de la précision des chiffres flottants.

In [ ]:
class EmBernoulli:
    def _init_center(self, data, nbComponents):
        center = np.zeros((data.shape[0], nbComponents))
        minimages = 0
        for i in range(nbComponents):
            mini = 0
            maxi = rd.randint(minimages, data.shape[1])
            center[:, i] = np.mean(data[:, mini:maxi], axis=1)

        return center

    def computeEM(self, data, nbComponents):
        delta = 1e-2
        # P(k)
        W = np.repeat(1 / nbComponents, nbComponents)
        center = self._init_center(data, nbComponents)
        while True:
            tabl = self._expectationStep(data, center, W)
            newW, center = self._maximizationStep(tabl, data)
            if np.sum(np.abs(newW - W)) < delta:
                return newW, center
            W = newW

    def _bernoulli(self, x, center):
        # proba is P(X1, ..., Xn)
        proba = np.zeros(x.shape)
        # Retrieve indices of black and white pixels.
        indices = [np.array(np.where(x == i)) for i in range(2)]
        proba[indices[0]] = 1 - center[indices[0]]
        proba[indices[1]] = center[indices[1]]
        prod = np.prod(proba)

        # This lines allow to fix NaN problems,
        # by replacing them by zeros
        if np.isnan(prod) or prod <= 0:
            return np.finfo(prod.dtype).eps
        return prod

    def _expectationStep(self, data, center, W):
        N = data.shape[1] # Number of data.
        D = data.shape[0] # Number of pixels.
        K = W.shape[0] # Number of classes.
        tabl = np.zeros((N, K))

        for n in range(N):
            for k in range(K):
                # Probability for the image to be from K class.
                tabl[n, k] = np.log(self._bernoulli(data[:, n], center[:, k])) + np.log(W[k])

            max_value = np.amax(tabl[n,:])
            tsum = np.sum(tabl[n, :] - max_value)

            #print((tabl[n, :] - max_value) - tsum)

            tabl[n, :] = np.exp((tabl[n, :] - max_value) - tsum)

        return tabl

    def _maximizationStep(self, tabl, data):
        N = tabl.shape[0] # Number of data.
        K = tabl.shape[1] # Number of classes.
        D = data.shape[0] # Number of pixels.
        tsum = np.sum(tabl, axis=0)
        W =  tsum / N
        center = np.dot(data, tabl) / (tsum)
        return W, center

In [ ]:
emb = EmBernoulli()
#W, center = emb.computeEM(test_data, 10)

**2.** Afficher les moyennes des différentes distributions de Bernoulli ?

In [ ]:
#vz.plotGroupImages(center)

**3.** En utilisant uniquement 10 composantes, est-il possible d'avoir un centre par classe ? Justifier.

In [ ]:
#TODO

**4.** Tester un classifieur bayésien en utilisant un mélange pour chaque classe ? Quel est le taux de reconnaissance obtenu (tester 1, 2, 4, et 8 composantes par classe) ?

In [ ]:
# TODO

## Partie 3 : Comparaison avec un GMM

Comparer les résultats obtenus avec le cas d'un mélange de gaussiennes sur les données brutes TRAIN et TEST.

* On a $N(X; \mu, \Theta) = \frac{1}{\sqrt{(2\pi)^D|\Theta|}} exp(-0.5(X - \mu)^T\Theta^{-1}(X - \mu))$
* La vraisemblance : $P(k|X) = \frac{w_k N(X;\mu_k,\Theta_k)}{\sum_{j=1}^{N}{w_j N(X,\mu_j,\Theta_j)}}$
* $P(k) = \frac{\sum_{i=1}^{N}{P(k|X_i)}}{N}$
* $\mu_k = \frac{\sum_{i=1}^{N}{P(k|X_i)X_i}}{\sum_{i=1}^{N}{P(k|X_i)}}$
* $\Theta_k = \frac{\sum_{i=1}^{N}{P(k|X_i)(X_i - \mu_k)^2}}{\sum_{i=1}^{N}{P(k|X_i)}}$

**1.** Envisager le cas de gaussienne avec des matrices de covariance diagonale, ensuite tester la version avec matrices de covariance complètes.

In [ ]:
class EmGaussian:
    def _init_center_cov(self):
        self.center = np.zeros((self.data.shape[0], self.nbComponents))
        #cov = [np.zeros((data.shape[0], data.shape[0])) for i in
        #        range(nbComponents)]
        self.cov = [np.identity(self.data.shape[0]) for i in
                    range(self.nbComponents)]
        minimages = self.data.shape[1] // 4

        for i in range(self.nbComponents):
            maxi = rd.randint(minimages, self.data.shape[1])
            self.center[:, i] = np.mean(self.data[:, minimages:maxi], axis=1)
            #cov[i] = np.cov(data[:, minimages:maxi])


    def __init__(self, data, nbComponents):
        self.data = data
        self.nbComponents = nbComponents
        self.delta = 1e-2

    def computeEM(self):
        # P(k)
        self.W = np.repeat(1 / self.nbComponents, self.nbComponents)
        self._init_center_cov()

        while True:
            self._expectationStep()
            oldW = self.W
            self._maximizationStep(tabl, center, data)
            if np.sum(np.abs(W - oldW)) < self.delta:
                return self.W, self.center, self.cov

    def _gaussian(self, x, k):
        # Here, we have to use the logarithm
        # in order to store the value in a double
        D = x.shape[0]
        xcentered = x - self.center[:, k]
        covdet = np.linalg.det(self.cov[k])
        covinv = np.linalg.pinv(self.cov[k])
        #piN = (2 * np.pi) ** D
        #denom = np.sqrt(piN * covdet)
        a = -0.5 * xcentered.T.dot(covinv).dot(xcentered)
        return -0.5 * (D * np.log(2 * np.pi) * np.log(covdet)) + a
        #return np.exp(-0.5 * xcentered.T.dot(covinv.dot(xcentered))) / denom

    def _expectationStep(self):
        N = self.data.shape[1] # Number of data.
        D = self.data.shape[0] # Number of pixels.
        K = self.W.shape[0] # Number of classes.
        self.tabl = np.zeros((N, K))
        # Likelihood
        for n in range(N):
            for k in range(K):
                # print("EXPECTATION n =", n, "k =", k)
                # Probability for the image to be from K class.
                self.tabl[n, k] = self._gaussian(self.data[:, n], k) * self.W[k]
            tsum = np.sum(self.tabl[n, :])
            self.tabl[n, :] = self.tabl[n, :] / tsum

    def _maximizationStep(self):
        print("MAXIMIZATION")
        N = self.tabl.shape[0] # Number of data.
        K = self.tabl.shape[1] # Number of classes.
        D = self.data.shape[0] # Number of pixels.
        tsum = np.sum(self.tabl, axis=0)
        oldcenter = W
        # Weights
        self.W =  tsum / N
        # Means
        self.center = np.dot(self.data, self.tabl) / tsum
        # Covariances
        xcentered = np.power(self.data - oldcenter, 2)
        self.cov = np.dot(xcentered, self.tabl) / tsum
        print(self.cov.shape)

In [ ]:
#emg = EmGaussian(test_data, 10)
#W, center, cov = emg.computeEM()
#vz.plotGroupImages(center)

**2.** Attention à la dégénérescence de l'algorithme EM (matrice de covariance non inversible). Il faut voir comment gérer ce problème ainsi que l'initialisation.

In [ ]:
# TODO

**3.** Proposer la meilleure solution pour avoir le plus haut taux de reconnaissance.

In [ ]:
# TODO